<center><h1><strong> Informe Tarea 1 - Beer Clustering </strong></h1></center>

<center><h3> Matías Pacheco &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;201204619-k</h3></center>
<center><h3> Juan Pablo Castillo Vera &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 201204595-9</h3></center>

## Descripción del dataset

El dataset Beer Reviews es una colección de 1.590.892 críticas de cervezas que se encuentran disponibles en el sitio http://beeradvocate.com/. Éstas críticas toman en consideración características como aroma, apariencia y sabor para asignarle un puntaje a la cerveza que está siendo evaluada.

Cada registro del dataset completo está compuesto por las siguientes columnas:

|Columna             |Significado            |Tipo       |
|---                 |---                    |---        |
|brewery_id          |ID cervecería          |Numérico   |
|brewery_name        |Nombre Cervecería      |Texto      |
|review_time         |Tiempo de evaluación   |Numérico   |
|review_overall      |Puntaje del review     |Numérico   |
|review_aroma        |Puntaje aroma          |Numérico   |
|review_appearence   |Puntaje apariencia     |Numérico   |
|review_profilename  |Nombre del evaluador   |Texto      |
|beer_style          |Tipo de cerveza        |Texto      |
|review_palate       |Puntaje paladar        |Numérico   |
|review_taste        |Puntaje gusto          |Numérico   |
|beer_abv            |Grado alcohólico       |Numérico   |
|beer_beerid         |ID cerveza             |Numérico   |

Cabe señalar que como pre-procesamiento de los datos, se setearon en cero todos aquellos campos numéricos del dataset que estuvieran vacíos (string '').


## Análisis de Clustering

Se realizará un análisis de clustering con el dataset con cada uno de los siguientes cinco algoritmos:
* K-Means
* Minibatch K-Means
* HAC Complete
* Ward
* DBScan

### Observaciones y Supuestos

1. Para reducir la dimensionalidad de los vectores, y posteriormente poder graficar los resultados en un minifold 2D, se utilizó Principal Component Analysis (PCA).

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

dataset = pd.read_csv('beer_reviews.csv')
...
reviews = PCA(n_components = 2).fit_transform(reviews)

2. Se tomaron los campos **brewery_id** y **beer_beerid** como las formas numéricas de las etiquetas **brewery_name** y **beer_type**. De esta forma, **brewery_id** es equivalente a **brewery_name**, y **beer_beerid** es equivalente a **beer_type**. 

3. Todos los análisis que se muestran a continuación se realizaron tomando los campos **brewery_id** y **beer_beerid**, y realizando el proceso de clustering considerando las siguientes 6 variables enteras del dataset: review_overall, review_aroma, review_appearance, review_palate, review_taste, beer_abv.

In [ ]:
reviews = pd.DataFrame({'brewery_id':dataset['brewery_id'],'review_overall':dataset['review_overall'],
                      'review_aroma':dataset['review_aroma'],'review_appearance':dataset['review_appearance'],
                      'review_palate':dataset['review_palate'],'review_taste':dataset['review_taste'],'beer_abv':dataset['beer_abv']})
    
reviews = pd.DataFrame({'beer_beerid':dataset['beer_beerid'],'review_overall':dataset['review_overall'],
                      'review_aroma':dataset['review_aroma'],'review_appearance':dataset['review_appearance'],
                      'review_palate':dataset['review_palate'],'review_taste':dataset['review_taste'],'beer_abv':dataset['beer_abv']})

4. En este informe se mostrarán sólo los gráficos resultantes del análisis del dataset con cada algoritmo. El código fuente está disponible en https://github.com/matipacheco/Tarea1-MD.


### K-Means

Los parámetros de K-Means utilizados para realizar el análisis fueron los siguientes:

* **n_init   = 10**. Se comprobó que incrementar la cantidad de veces que se ejecuta el algoritmo no mejora significativamente el resultado, sólo se produce un pequeño cambio en la posición de los centroides.
* **max_iter = 300 (default)**. Se comprobó que incrementar la cantidad máxima de iteraciones del algoritmo no mejora el resultado.
* **init     = 'k-means++'**. Para una convergencia más rápida.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

A continuación se muestran los resultados del clustering por *brewery_id* para 2, 3, 4 y 5 clusters:

<center><img src="PlotsKMeans/todas_0.png"></center>
<center><small>Figura 1: Clustering por brewery_id utilizando K-Means.</small></center>

Como se puede apreciar, no existe una segmentación clara para los clusters. Si bien existe un cluster a la derecha que separa del resto, aún así no existe información suficiente para determinar cuántos clusters son. 

Como se muestra en la siguiente figura, para k = 6 clusters, uno de los centroides queda sobre el espacio vacío que separa los elementos de la izquierda y derecha, lo que indica que la segmentación con al menos esta cantidad de clusters, no se adecúa correctamente a la muestra. De esta forma, a partir de los datos recopilados se determina que k = 5 es el número indicado de clusters que representa a la muestra, ya que los centroides quedan bien repartidos a lo largo de la muestra, como se ve en la imagen.

<center><img src="PlotsKMeans/kmeans_0_6clusters.png"></center>
<center><small>Figura 2: Clustering por brewery_id utilizando K-Means con 6 clusters.</small></center>

A continuación se muestran los resultados del clustering por *beer_beerid* para 3, 4, 5 y 6 clusters:

<center><img src="PlotsKMeans/todas_1.png"></center>
<center><small>Figura 3: Clustering por beer_beerid utilizando K-Means.</small></center>

Si bien al clusterizar por brewery_id se notaban **al menos** dos clusters claramente separados, en este caso no existe ninguna separación o diferencia entre los elementos de la imagen. Pero resulta claro que los elementos no corresponden a un único gran cluster. A partir de la pruebas realizadas, con k = 5 o k = 6 clusters la muestra queda bien representada, como se puede ver en la figura anterior.


### Minibatch K-Means

Los parámetros de Minibatch K-Means utilizados para realizar el análisis fueron los siguientes:

* **n_init   = 100**. Se comprobó que incrementar la cantidad de veces que se ejecuta el algoritmo produce mejoras en el posicionamiento de los centroides.
* **batch_size = 1000**. Se comprobó que incrementar el tamaño de los batches, mejora el posicionamiento de los centroides de los clusters, pero para valores mayores a 1000 este cambio es muy poco significativo.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

A continuación se muestran los resultados del clustering por *brewery_id* para 2, 3, 4 y 5 clusters:

<center><img src="PlotsMinibatchKMeans/todas_0.png"></center>
<center><small>Figura 4: Clustering por brewery_id utilizando Minibatch K-Means.</small></center>

Como se puede apreciar, la figura obtenida es muy similar a la que se obtuvo para el análisis con K-Means, lo que demuestra lo similar que son ambos algoritmos. Dicho esto, el análisis que se hizo para K-Means también se aplica para este caso, por lo que se obtienen k = 5 clusters que describen el dataset.

De la misma forma, a continuación se muestran los resultados del clustering por *beer_beerid* para 3, 4, 5 y 6 clusters:

<center><img src="PlotsMinibatchKMeans/todas_1.png"></center>
<center><small>Figura 5: Clustering por beer_beerid utilizando Minibatch K-Means.</small></center>

Al igual que para K-Means, para Minibatch K-Means k = 5 o k = 6 describen correctamente la muestra.


### HAC Complete

En primer lugar, es necesario aclarar que para realizar los análisis con el algoritmo HAC, se utilizó un subconjunto de 20.000 elementos del dataset, a diferencia de los dos algoritmos anteriores, en los que se analizó el dataset completo. Esto se debe a que para un número mayor de elementos la ejecución del algoritmo consume muchísimos recursos del computador, y por falta de hardware y memoria el algoritmo siempre terminaba cayéndose.

Los parámetros de HAC utilizados para realizar el análisis fueron los siguientes:

* **linkage   = 'complete'**. Por restricción de la tarea.
* **affinity = 'euclidean'**. Se comprobó que sólo con la métrica cosine se obtenían clusters distintos. Las métricas euclidean, manhattan, l1 y l2 formaban los mismos clusters.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Para este algoritmo, como no tiene algunos de los atributos que sí tiene K-Means y Minibatch K-Means, la graficación de los clusters se debe hacer de manera un poco distinta, pero se tiene la ventaja de que para este algoritmo, se pueden colorear los clusters para diferenciarlos de mejor manera.  A continuación se muestran los resultados del clustering por *brewery_id* para 2, 3, 4 y 5 clusters:

<center><img src="PlotsHAC/todas_0.png"></center>
<center><small>Figura 6: Clustering por brewery_id utilizando HAC.</small></center>

De la misma forma, a continuación se muestran los resultados del clustering por *beer_beerid* para 2, 3, 4 y 5 clusters:

<center><img src="PlotsHAC/todas_1.png"></center>
<center><small>Figura 7: Clustering por beer_beerid utilizando HAC.</small></center>

Como se puede ver, aún cuando para éste algoritmo se tomó una muestra del 0.1% del dataset original, ambas gráficas son muy similares a las obtenidas para K-Means y Minibatch K-Means, para ambos clusterings (*brewery_id* y *beer_beerid*). Sin embargo, en este caso resulta difícil determinar una cantidad adecuada de clusters cuando de segmenta por *brewery_id*, mientras que para el clustering por *beer_beerid* se nota claramente que con k = 5 se obtiene una cantidad de clusters que describen bastante bien la distribución de los elementos de la muestra.


### Ward


### DBScan



## Conclusiones

¿Qué atributo, Nombre de cervecería o Tipo de cerveza, describe mejor a los cluster como etiquetas
de clase, según los resultados obtenidos previamente? ¿Hay mejores marcas que otras en
relación a las evaluaciones obtenidas? ¿Se puede definir algún criterio para determinar el mejor
tipo de cerveza? Comente